### **Import Dependencies**

In [ ]:
import json
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
import torch

### **Data Pre-processing**

In [ ]:
import json

dataset_path = "/content/intent.json"
with open(dataset_path, "r") as file:
    data = json.load(file)

intents = data["intents"]
questions = []
responses = []
for intent in intents:
    for pattern in intent["patterns"]:
        questions.append(pattern)
        # Append each response individually
        for response in intent["responses"]:
            responses.append(response)

### **Loading the dataset and tokenizing the Input**

In [ ]:
import json
import torch
from transformers import BertForSequenceClassification, BertTokenizer

# Load the intents dataset
dataset_path = "/content/intent.json"
with open(dataset_path, "r") as file:
    data = json.load(file)

# Extract questions and responses from intents
questions = []
responses = []
for intent in data["intents"]:
    questions.extend(intent["patterns"])
    responses.extend(intent["responses"])

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Define a function to chat with the bot
def chat_with_bot():
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("Chatbot: Goodbye!")
            break
        # Tokenize user input
        inputs = tokenizer(user_input, padding=True, truncation=True, return_tensors="pt")
        # Get predicted label using BERT model
        with torch.no_grad():
            outputs = model(**inputs)
        predicted_label = torch.argmax(outputs.logits)
        # Print corresponding response
        print("Chatbot:", responses[predicted_label.item()])

# Start chatting
chat_with_bot()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


You: Hii
Chatbot: Hi there, what can I do for you?
You: Quit
Chatbot: Goodbye!


### **Model Training**

In [ ]:
from torch.utils.data import DataLoader, Dataset
from transformers import BertForSequenceClassification, BertTokenizer
from sklearn.model_selection import train_test_split
import torch

# Define a custom dataset class
class IntentDataset(Dataset):
    def __init__(self, questions, responses, tokenizer, max_length=128):
        self.questions = questions
        self.responses = responses
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        response = self.responses[idx]

        encoding = self.tokenizer(question, response,
                                  padding='max_length',
                                  truncation=True,
                                  max_length=self.max_length,
                                  return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(1)  # We use a constant label since this is not a classification task
        }

# Create train and validation datasets
train_questions, val_questions, train_responses, val_responses = train_test_split(questions, responses, test_size=0.1, random_state=42)
train_dataset = IntentDataset(train_questions, train_responses, tokenizer)
val_dataset = IntentDataset(val_questions, val_responses, tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Fine-tune the BERT model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

for epoch in range(10):
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

    val_loss /= len(val_loader)
    print(f'Epoch {epoch + 1}, Validation Loss: {val_loss:.4f}')

print("Training finished!")


Epoch 1, Validation Loss: 0.3121
Epoch 2, Validation Loss: 0.1225
Epoch 3, Validation Loss: 0.0645
Epoch 4, Validation Loss: 0.0416
Epoch 5, Validation Loss: 0.0227
Epoch 6, Validation Loss: 0.0159
Epoch 7, Validation Loss: 0.0106
Epoch 8, Validation Loss: 0.0074
Epoch 9, Validation Loss: 0.0061
Epoch 10, Validation Loss: 0.0050
Training finished!


In [ ]:
# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/Model/berta.pth")

In [ ]:
torch.save(model, "/content/drive/MyDrive/Model/berta.pth")

In [ ]:
model = torch.load('/content/drive/MyDrive/chatbot_models/Berth.pth', map_location=torch.device('cpu'))

In [ ]:
from transformers import BertForSequenceClassification

# Load the saved model from the directory
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/chatbot_models/Berth.pth")



### **Chatbot**

In [ ]:
import random
import json

import torch

from model import transformer
from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intent.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "/content/intent.json"
data = torch.load(FILE)

bot_name = "Chatbot"
print("Let's chat! (type 'quit' to exit)")
while True:
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")



Let's chat! (type 'quit' to exit)
You: How can I get in touch with your support team?
Chetana: Interested in bulk ordering? Contact our support team for more information.
You: How do I report a problem with my order?
Chetana: Forgot your password? Contact our support team for assistance with account recovery.
You: Where can I find help with a technical issue?
Chetana: If you received the wrong item, our support team will assist you with exchanging it.
You: How can I contact customer support?
Chetana: Access your digital purchases by contacting our support team.
You: I need help with my account.",
Chetana: Trouble accessing your order history? Contact our support team for assistance.
You: What's the best way to get help with a problem?
Chetana: Trouble applying for a refund through the mobile app? Contact our support team for assistance.
You: Is there a phone number I can call for assistance?",
Chetana: Set preferences for order notifications by contacting our support team.
You: Is ther

In [ ]:
!pip install --upgrade nltk


### **Model Evaluation**

In [29]:
# Define a function to evaluate the model
import nltk
from nltk.tokenize import word_tokenize
import json
import numpy as np
import torch

nltk.download('punkt')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def tokenize(sentence):
    return word_tokenize(sentence)

def bag_of_words(sentence, words):
    bag = np.zeros(len(words), dtype=np.float32)
    for word in sentence:
        if word in words:
            bag[words.index(word)] = 1
    return bag


def evaluate_model(model, test_data, intents):
    num_correct = 0
    total = 0

    for example in test_data:
        sentence = example['sentence']
        expected_tag = example['tag']

        sentence = tokenize(sentence)
        X = bag_of_words(sentence, all_words)
        X = X.reshape(1, X.shape[0])
        X = torch.from_numpy(X).to(device)

        # Convert input tensor to torch.long
        X = torch.tensor(X, dtype=torch.long, device=device).clone().detach()

# Call the model for inference
        output = model(X)

        # Extract logits from the output
        logits = output.logits

# Apply torch.max() to find the index of the maximum value along the specified dimension
        _, predicted = torch.max(logits, dim=1)
        # _, predicted = torch.max(output, dim=1)

        tag = tags[predicted.item()]

        # print(tag, expected_tag)
        if tag == expected_tag:
            num_correct += 1

        total += 1

    accuracy = num_correct / total
    print(f"Accuracy: {accuracy * 100:.2f}%")

# Load the test dataset
test_data = [
    {"sentence": "Hi there!", "tag": "greeting"},
    {"sentence": "What kinds of items do you have?", "tag": "items"},
    {"sentence": "You're welcome! Happy to assist.", "tag": "goodbye"},
    # {"sentence": "How are you", "tag": "greeting"},
    {"sentence": "Hi there! How's everything been going with you lately?", "tag": "greeting"},
    {"sentence": "Bye! Come back again soon.", "tag": "goodbye"},
    {"sentence": "No problem! Glad I could help.", "tag": "goodbye"},
    {"sentence": "Bubye. Let's meet again soon", "tag": "goodbye"},
    # {"sentence": "Delivery typically takes 2-4 business days.", "tag": "delivery"},
    {"sentence": "You're welcome! Happy to assist. Goodbye", "tag": "goodbye"},
    {"sentence": "Hello!! I am glad I could help you", "tag": "goodbye"},
    {"sentence": "Bye. Have a good day", "tag":"goodbye"},
    {"sentence": "Who can I talk to about a refund", "tag": "support"}

]

with open('/content/intent.json', 'r') as json_data:
    intents = json.load(json_data)

# Assuming your training data is stored in a variable called intents
all_words = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        words = tokenize(pattern)
        all_words.extend(words)

# Extracting unique tags from the intents data
tags = []
for intent in intents['intents']:
    tags.append(intent['tag'])


# Remove duplicates
all_words = list(set(all_words))


# Call the evaluate_model function
evaluate_model(model, test_data, intents)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-29-693631277ead>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.long, device=device).clone().detach()


Accuracy: 63.64%
